# 1. Import dependencies

In [7]:
#Modules en lien avec Gym
import gymnasium as gym

#Modules utiles
import sys
sys.path.append('/Users/mathiasperez/Documents/GitHub/PSC_Stratolia/Algo Naif')
sys.path.append('/Users/mathiasperez/Documents/GitHub/PSC_Stratolia')
from data_vent import ventU_ventV, ventU_ventV_interpolate, distance_destination , convPression_altitude
import Node
import RL.Balloon as Balloon

import os
import numpy as np
import random
import os
import datetime as dt 
import time
import math

"""
#Modules en lien avec Stable_baseline3
#IMPORT THE TYPE OF AGENT U WANT TO USE
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
"""

FileNotFoundError: [Errno 2] No such file or directory: 'objet_wind_data_2020.pickle'

# 1.bis Import the wind-datas

In [3]:
import pickle
with open("objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)

"""
Le tableau va de :
datetime.datetime(2020, 1, 1, 0, 0) à datetime.datetime(2021, 1, 1, 0, 0)
"""

FileNotFoundError: [Errno 2] No such file or directory: 'objet_wind_data_2020.pickle'

In [1]:
print((wind_data['metadata']['grid']))

NameError: name 'wind_data' is not defined

# 1.bisbis Create needed functions

In [5]:
"""
    INPUT : start_date and end_date -> datetime objects representing the start and end dates
    OUTPUT : A random date bewtween the two inputs and an hour  that is a multiple of 6 (i.e., 0, 6, 12, or 18)
"""
def generate_random_date(start_date, end_date):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + dt.timedelta(days=random_number_of_days)
    hr  = random.randrange(0, 24, 6)
    random_time = dt.time(hour=hr, minute=0, second=0, microsecond=0, tzinfo=None, fold=0)
    return dt.datetime.combine(random_date, random_time)


# 2. Create the environnement

In [8]:
class BalloonEnv(gym.Env):
    """
    ## Description
    This environment is a complete representation of our problem.

    The destination is always at variable coordinates (random opr not).
    So are the starting coordinatesv (probably)


    ## Action Space
    There are three discrete actions available:
    - 0: do nothing et let the wind lead your movement
    - 1: go up in altitude (17 heights allowed)
    - 2: go down in altitude (17 heights allowed)

    ## Observation Space
    The state a tuple consisting of different parts : 
        First you have the time accessible using the datetime module
        Then 2 Boxes (simple array) for the longitude and the latitude
        Finally one discrete part consisting of the 71 different heights

    ## Rewards
    After every step a reward is granted. The total reward of an episode is the
    sum of the rewards for all the steps within that episode.

    For each step, the reward:
    (- is increased/decreased the closer/further the ballon is to the goal point.)
    - is increased/decreased the slower/faster the lander is moving.
    (- is increased/decreased the less/more energy from the battery is used in the round)

    The episode receive an additional reward of +100 or -100 points for finding the goal or not respectively.



    ## Starting State
    The lander starts at a random poinf of the globe (time can be random as well)

    ## Episode Termination
    The episode finishes if:
    1) the balloon gets too far away from the goal;
    2) the balloon goes near enough of the goal

    ## Arguments

     * `goal_long` and 'goal_lat' for the position of our goal destination (can be fixed for specific results)
     * 'initial_time' defines the starting time of our episode (=> consequence on the winds)

    * `precision' defines how close the balloon must be to the target before being considered as having reached it.

    * 'limite_eloignement' defines how far the ballon must be from the target to consider the episode to be a failure.

    * `temps_exploration` defines how long we let the ballon follow the winds when action : 0 is considered

    ## Version History
    - v0: Initial version

    ## Current bugs

    ## Credits
    Mathias PEREZ et ses sbires
    """
    def __init__(self):
        #super(Balloon, self).__init__()
        self.step : int

        self.basStrat : Balloon

        self.initial_time : dt.datetime

        self.limite_eloignement = 10000 #en m
        self.precision = 1000 #en m

        self.temps_exploration = 131 #secondes

        self.trajectoire : list
        
        # Define the action space
        self.action_space = gym.spaces.Discrete(3)  
        """
        Action 0 : You do nothing and let the balloon follow the wind for temps_exploration seconds
        Action 1 : You push the balloon upwards (goes up in altitude) (instanneous action)
        Action 2 : You push the balloon downwards (goes down in altitude) (instanneous action)
        """
        
        taille_tab_vent = 13

        # Define the observation space
        #firt we have position of the balloon, then the goal positon then the local wind data
        low_obs = np.array([0, -90, 0, -90] + [-100, -50] * (taille_tab_vent))
        high_obs = np.array([360,90,360,+90] + [100, 50] * (taille_tab_vent))
        self.observation_space = gym.spaces.Tuple(( 
            gym.spaces.Box(low=low_obs, high=high_obs, dtype=np.float32), 
            gym.spaces.Discrete(17), #Altitude (17 different positions : no need for more precisions)
            ))
     
    def reset(self, tab_vent : dict):
        # Reset the environment to its initial state
        # Resetting the ballon to a random initial position
        #Ici on pourrait randomiser les goal et les positions de depart +temps
        self.initial_time = generate_random_date(dt.datetime(2020, 1, 1, 0, 0), dt.datetime(2021, 1, 1, 0, 0))
        self.step = 0
        
        goal_long, goal_lat = random.uniform(0,360), random.uniform(-90,90)
        depart_long, depart_lat = random.uniform(0,360), random.uniform(-90,90)
        while distance_destination((self.goal_long, self.goal_lat), depart_long, depart_lat) > 10000000 :
            goal_lat = random.uniform(-90,90)
            goal_long = random.uniform(0,360)
        
        self.basStrat = Balloon(depart_long, depart_lat, self.initial_time, 16, None, precision = 1000)
        self.trajectoire = []
        
        tab1 = [depart_long, depart_lat, goal_long, goal_lat]
        tab2inter = self.basStrat.get_winds(tab_vent)
        tab2 = []
        for i in range(len(tab2inter)):
            tab2.append(tab2inter[i][0])
            tab2.append(tab2inter[i][1])

        self.state = (
                      np.concatenate(tab1, tab2),
                      16
                      )
        return self.state, {}
    
    def get_terminated(self, state):
        long, lat = state[0][0], state[0][1]
        if distance_destination((self.goal_long, self.goal_lat), long, lat) < self.limite_eloignement :
            return True
        return False
    
    def get_reward(self, state):
        """
        - fonction qui donne récompense en fonction de la distance  a l'objectif par rapport a la distance initale
        - ajoute un boolean si l'objectif a été atteint ou bien si on en est suffisamment proche
        - d'autres trucs a faire surement
        """
        eloignement = distance_destination((self.state[0][2], self.state[0][3]), state[0][0], state[0][1])
        time  =  self.basStrat.t.total_seconds() - self.initial_time.total_seconds()
        if (eloignement <= self.limite_eloignement) :
            reward =  1000 + math.log(1/time) - self.step
        else :
            reward = math.exp((self.limite_eloignement/eloignement)) +  math.log(1/time) - self.step
        return reward

    #### RENDER A RAJOUTER ICI
    def step(self, action):
        self.step += 1
        temps = (self.basStrat.t-dt.datetime(2020, 1, 1, 0, 0)).total_seconds()
        self.trajectoire.append([self.basStrat.long,self.basStrat.lat, convPression_altitude(self.basStrat.p), temps ])
        # Take a step in the environment based on the given action
        if action == 0:
            #You decide to follow the wind flow :
            done = self.basStrat.follow_wind(self.temps_exploration, self.state[0][2], self.state[0][3], wind_data)
            self.state[0][0], self.state[0][1] = self.basStrat.long, self.basStrat.lat

        elif action == 1:
            #You go up 1 case in altitude so u go down in pressure
            done  = self.basStrat.up(self.temps_exploration,self.state[0][2], self.state[0][3], wind_data)
            self.state[0][0], self.state[0][1] = self.basStrat.long, self.basStrat.lat #au cas ou on soit en limite haute
        else :
            #You go down one case in altitudeso u go up in pressure
            done  = self.basStrat.down(self.temps_exploration,self.state[0][2], self.state[0][3], wind_data)
            self.state[0][0], self.state[0][1] = self.basStrat.long, self.basStrat.lat #au cas ou on soit en limite basse

        # Calculate the reward
        reward = get_reward(self.state)
        
        # Check if the episode has ended
        if not done:
            done = get_terminated(self.state)
        
        # Check if the episode has been truncated
        truncated = False
        
        # Return the observation, reward, done flag, truncated flag, and info
        return self.state, reward, done, truncated, {}
    
    def render(self, mode='human'):
        pass


    


In [9]:
env = BalloonEnv()

env.observation_space

Tuple(Box([   0.  -90.    0.  -90. -100.  -50. -100.  -50. -100.  -50. -100.  -50.
 -100.  -50. -100.  -50. -100.  -50. -100.  -50. -100.  -50. -100.  -50.
 -100.  -50. -100.  -50. -100.  -50.], [360.  90. 360.  90. 100.  50. 100.  50. 100.  50. 100.  50. 100.  50.
 100.  50. 100.  50. 100.  50. 100.  50. 100.  50. 100.  50. 100.  50.
 100.  50.], (30,), float32), Discrete(17))